**Name:** \_\_\_\_\_Huanchen Wang\_\_\_\_\_

**EID:** \_\_\_\_\_57558749\_\_\_\_\_

**Kaggle Team Name:** \_\_\_\_\_unknownwang\_\_\_\_\_

# CS5489 - Assignment 1 - Movie Rating Prediction

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [1]:
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [2]:
def read_text_data(fname):
    txtdata = []
    revid   = []
    classes = []
    with open(fname, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            revid.append(row[1])
            # get the class (convert to integer)
            if len(row)>2:
                classes.append(int(row[2]))
    
    if (len(classes)>0) and (len(txtdata) != len(revid)):        
        warn.error("mismatched length!")
    
    return (txtdata, revid, classes)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [3]:
# load the data
(traintxt, trainrevid, trainY) = read_text_data("movierating_train.txt")
(testtxt, testrevid, _)        = read_text_data("movierating_test.txt")

print(len(traintxt))
print(len(testtxt))

4006
1000


In [ ]:
### INSERT YOUR CODE HERE

In [4]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
# nltk.download()
import datasets

In [5]:
def tokenize_word(doc):
    # get stopwords list
    stop_words = stopwords.words('english')
    # tokenize the text each word initially
    tokens = nltk.word_tokenize(doc)
    # remove the punctuation and stopwords in the text
    tokens = [word for word  in tokens if word not in stop_words and word not in punctuation]
    return ' '.join(tokens)

traintxt = [tokenize_word(doc) for doc in traintxt]
testtxt = [tokenize_word(doc) for doc in testtxt]

dataset= datasets.Dataset.from_dict({'text':traintxt, 'label':trainY})


In [6]:
# from datasets import load_metric
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
use_gpu = torch.cuda.is_available()
import evaluate
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True,max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                            num_labels=4)
training_args = TrainingArguments(output_dir="test_trainer", learning_rate=2e-5, per_device_train_batch_size=8, per_device_eval_batch_size=8, seed=46, 
                                  lr_scheduler_type='cosine', logging_dir="test_trainer", evaluation_strategy='epoch', metric_for_best_model='accuracy', greater_is_better=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [10]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
d:\Anaconda\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3605
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1353
  Number of trainable parameters = 108313348


  0%|          | 0/1353 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 401
  Batch size = 8


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 1.0684934854507446, 'eval_accuracy': 0.49625935162094764, 'eval_runtime': 5.3324, 'eval_samples_per_second': 75.201, 'eval_steps_per_second': 9.564, 'epoch': 1.0}


...

Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 401
  Batch size = 8


  0%|          | 0/51 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.8893514275550842, 'eval_accuracy': 0.6209476309226932, 'eval_runtime': 5.5964, 'eval_samples_per_second': 71.654, 'eval_steps_per_second': 9.113, 'epoch': 3.0}
{'train_runtime': 458.6118, 'train_samples_per_second': 23.582, 'train_steps_per_second': 2.95, 'train_loss': 0.9053627685185988, 'epoch': 3.0}


TrainOutput(global_step=1353, training_loss=0.9053627685185988, metrics={'train_runtime': 458.6118, 'train_samples_per_second': 23.582, 'train_steps_per_second': 2.95, 'train_loss': 0.9053627685185988, 'epoch': 3.0})

In [12]:
testdataset = datasets.Dataset.from_dict({'text':testtxt})

test_tokenized_datasets = testdataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
test_labels = trainer.predict(test_tokenized_datasets).predictions.argmax(-1)
test_labels

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


  0%|          | 0/125 [00:00<?, ?it/s]

array([2, 1, 2, 3, 2, 2, 3, 1, 1, 3, 1, 1, 1, 2, 3, 2, 1, 1, 2, 3, 3, 1,
       0, 2, 3, 1, 1, 1, 3, 3, 2, 3, 1, 1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 2,
       0, 1, 3, 1, 1, 1, 1, 2, 2, 2, 1, 3, 1, 0, 1, 1, 3, 1, 3, 1, 3, 2,
       0, 2, 2, 1, 3, 2, 0, 1, 3, 1, 1, 0, 1, 2, 0, 2, 2, 1, 3, 0, 1, 2,
       2, 3, 0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 2,
       1, 2, 1, 2, 2, 3, 2, 2, 1, 1, 1, 2, 1, 3, 3, 2, 2, 2, 3, 2, 3, 2,
       1, 1, 2, 3, 1, 3, 3, 1, 1, 0, 2, 2, 0, 2, 3, 2, 2, 3, 1, 2, 1, 1,
       0, 3, 1, 1, 1, 1, 1, 0, 3, 2, 1, 0, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2,
       1, 2, 0, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 1, 2, 2, 1, 2, 1,
       1, 1, 0, 0, 3, 2, 1, 2, 1, 2, 1, 2, 3, 2, 2, 1, 1, 3, 0, 1, 2, 2,
       2, 0, 1, 0, 3, 1, 1, 2, 1, 2, 2, 2, 2, 0, 2, 1, 3, 2, 2, 1, 1, 1,
       3, 2, 3, 2, 2, 3, 1, 1, 2, 2, 1, 1, 3, 3, 0, 2, 3, 2, 3, 2, 1, 3,
       3, 1, 2, 1, 2, 0, 1, 2, 1, 1, 3, 2, 2, 0, 2, 0, 1, 2, 2, 3, 2, 1,
       1, 2, 0, 2, 2, 1, 2, 1, 2, 2, 1, 3, 2, 1, 1,

In [14]:
test_labels = test_labels.tolist()
write_csv_kaggle_sub("my_submission.csv", test_labels)